### ライブラリのインポート

In [24]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import os
import math
import optuna
import re

### 計算用関数

In [25]:
# 行数を指定すると所用時間を返す関数
def knapsack(row_x, data_x,required_time_x, waiting_time_x, move_time_x, attraction_no_x):
    waiting_time_x = data_x.iloc[row_x, attraction_no_x]
    ans = required_time_x + waiting_time_x + move_time_x
    return ans


# 合計所要時間と合計人気度を算出する関数
def ride_calculation(data, ride_lst, required_times, popularity, move_time):
    # print(data)
    # print(ride_lst)
    # print(required_times)
    # print(move_time)
    total_popularity = 0
    total_time = 0
    ride_time = []
    new_ride_lst = []
    # start_time = data.index[0]
    # print("start time is:", calculate_time(start_time))
    row = 0
    try:
        for i in ride_lst:
            # total_timeが675以上かつiが0または1の場合にループを強制終了
            # ソアリン、トイストーリーMは20:15以降DPA所持者のみ乗車可能
            if (data.index[row]) >= 675 and (i == 0 or i == 1):
                continue
            
            else:
                # iが8以上の場合、次のループへ(該当しないアトラクション)
                if i >= 8 or i < 0:
                    # print("不明なアトラクション：", i)
                    continue
                else:
                    # print("-" * 40)
                    # print(calculate_time(data.index[row]))
                    ride_time.append(data.index[row])
                    # print(attractions.get(i))
                    # print("移動時間", move_time)
                    waiting_time = data.iloc[row, i]
                    # print("待ち時間:", waiting_time)
                    required_time = required_times[i]
                    # print("観賞時間:", required_time)
                    ans = knapsack(row, data, required_time, waiting_time, move_time, i)
                    # print("合計所要時間:", ans)
                    total_time += ans
                    row = int(np.ceil(total_time / 15))
                    total_popularity += popularity[i]
                    new_ride_lst.append(i)
    except Exception as e:
        # print("全てのアトラクションを乗ることができません")
        pass

    # print("-" * 40)
    # print(total_popularity)
    return total_popularity, total_time, new_ride_lst, ride_time

In [26]:
#initial value.
lamb=1.0
tt_best=720
j=0
NN=30
#while((j<15)& (tt_best>660)):
#while((j<15)& (np.abs(tt_best-500)>160)):
while(np.abs(tt_best-615)>60):
#for j in range(2):
    print(tt_best)
    total_time_list=[]
    def objective(trial):
        # アトラクションの所要時間と人気度
        required_times = [5, 7, 2, 3, 3, 2, 23, 30]  # 所要時間
        popularity = [84, 69, 55, 45, 33, 28, 19, 22]  # 人気度
        move_time = 15

        file_path = '0724/2022-07-24.csv'
        # print(file_path)
        data = pd.read_csv(file_path)
        data = data.set_index('時間')

        # 乗り物の数を決定
        n_rides = trial.suggest_int('n_rides', 1, 20)  # 最大15の乗り物を仮定

        # 乗り物の定義
        ride_lst = [trial.suggest_int(f'ride_{i}', 0, 7) for i in range(n_rides)]

        # ride_calculation関数の呼び出し
        total_popularity, total_time, new_ride_lst, ride_time = ride_calculation(data, ride_lst, required_times, popularity, move_time)

        # 制約条件の範囲
        max_allowed_time = 660  # パーク内にいる時間 9:00~20:00 計660分 (20時以降はファストパスのみ受け入れと仮定)
        #trial.set_user_attr("constraints", [1000*(max_allowed_time - total_time)])
        print("total_time:",total_time)
        print("total_popularity:",total_popularity)
        total_time_list.append(total_time)
        return total_popularity+lamb*(max_allowed_time - total_time)
    
    # Optunaによる最適化
    study = optuna.create_study(direction="maximize")  # 人気度を最大化
    study.optimize(objective, n_trials = NN)
    best_trial = study.best_trial
    tt_best = total_time_list[best_trial.number] #最適解における所要時間の値。これをペナルティに用いる。
    
    best_trial = study.best_trial
    #print(f"Best trial: {best_trial.number}")
    #print(f"Best popularity: {best_trial.value}")
    #print(f"Parameters: {best_trial.params}")
    #print("total_time here is:", tt_best)
    
    
    lamb_list=[]
    def objective_dual(trial_dual):
        #lamb_list=[]
        # アトラクションの所要時間と人気度
        required_times = [5, 7, 2, 3, 3, 2, 23, 30]  # 所要時間
        popularity = [84, 69, 55, 45, 33, 28, 19, 22]  # 人気度
        move_time = 15

        file_path = '0724/2022-07-24.csv'
        # print(file_path)
        data = pd.read_csv(file_path)
        data = data.set_index('時間')

        # 乗り物の数を決定
        n_rides = trial_dual.suggest_int('n_rides', 1, 20)  # 最大15の乗り物を仮定

        # 乗り物の定義
        ride_lst = [trial_dual.suggest_int(f'ride_{i}', 0, 7) for i in range(n_rides)]

        # ride_calculation関数の呼び出し
        total_popularity, total_time, new_ride_lst, ride_time = ride_calculation(data, ride_lst, required_times, popularity, move_time)

        # 制約条件の範囲
        max_allowed_time = 660  # パーク内にいる時間 9:00~20:00 計660分 (20時以降はファストパスのみ受け入れと仮定)
        #lamb_x = 0.8*trial_dual.suggest_int('n_rides', 1, 3)
        lamb_x = trial_dual.suggest_float('lamb_x', 0.5, 2)
        #print("lamb:",lamb_x)
        lamb_list.append(lamb_x)
        #print("total_time:",total_time)
        #print("total_popularity:",total_popularity)
        penalty=max(0,(max_allowed_time - tt_best))
        #print("penalty",penalty)
        return total_popularity+lamb_x*penalty
    
    
    study_dual = optuna.create_study(direction="minimize")  # 人気度を最小化
    study_dual.optimize(objective_dual, n_trials = NN)
    
    
    best_trial_dual = study_dual.best_trial
    lamb = lamb_list[best_trial_dual.number]
    j=j+1
    print("tt_best2:",tt_best)
    #print("lamb is",lamb)

[I 2024-10-18 21:43:54,829] A new study created in memory with name: no-name-8f36ea0f-c0b8-4f86-b5ae-fb8402e7ac93
[I 2024-10-18 21:43:54,839] Trial 0 finished with value: 555.0 and parameters: {'n_rides': 19, 'ride_0': 3, 'ride_1': 6, 'ride_2': 0, 'ride_3': 0, 'ride_4': 0, 'ride_5': 0, 'ride_6': 6, 'ride_7': 0, 'ride_8': 6, 'ride_9': 2, 'ride_10': 0, 'ride_11': 3, 'ride_12': 6, 'ride_13': 0, 'ride_14': 7, 'ride_15': 6, 'ride_16': 4, 'ride_17': 6, 'ride_18': 3}. Best is trial 0 with value: 555.0.
[I 2024-10-18 21:43:54,839] Trial 1 finished with value: 528.0 and parameters: {'n_rides': 17, 'ride_0': 0, 'ride_1': 3, 'ride_2': 0, 'ride_3': 1, 'ride_4': 2, 'ride_5': 7, 'ride_6': 3, 'ride_7': 6, 'ride_8': 1, 'ride_9': 4, 'ride_10': 6, 'ride_11': 4, 'ride_12': 7, 'ride_13': 0, 'ride_14': 1, 'ride_15': 5, 'ride_16': 4}. Best is trial 0 with value: 555.0.
[I 2024-10-18 21:43:54,853] Trial 2 finished with value: 618.0 and parameters: {'n_rides': 2, 'ride_0': 7, 'ride_1': 5}. Best is trial 2 wit

720
total_time: 727
total_popularity: 622
total_time: 709
total_popularity: 577
total_time: 92
total_popularity: 50
total_time: 63
total_popularity: 45
total_time: 171
total_popularity: 164
total_time: 637
total_popularity: 529
total_time: 741
total_popularity: 686
total_time: 56
total_popularity: 66
total_time: 706
total_popularity: 567
total_time: 566
total_popularity: 421
total_time: 381
total_popularity: 218
total_time: 235
total_popularity: 149
total_time: 332
total_popularity: 224
total_time: 333
total_popularity: 262


[I 2024-10-18 21:43:55,035] Trial 14 finished with value: 625.0 and parameters: {'n_rides': 4, 'ride_0': 6, 'ride_1': 4, 'ride_2': 5, 'ride_3': 2}. Best is trial 7 with value: 670.0.
[I 2024-10-18 21:43:55,041] Trial 15 finished with value: 598.0 and parameters: {'n_rides': 1, 'ride_0': 2}. Best is trial 7 with value: 670.0.
[I 2024-10-18 21:43:55,098] Trial 16 finished with value: 511.0 and parameters: {'n_rides': 12, 'ride_0': 4, 'ride_1': 2, 'ride_2': 2, 'ride_3': 5, 'ride_4': 5, 'ride_5': 2, 'ride_6': 4, 'ride_7': 4, 'ride_8': 0, 'ride_9': 7, 'ride_10': 3, 'ride_11': 7}. Best is trial 7 with value: 670.0.
[I 2024-10-18 21:43:55,122] Trial 17 finished with value: 599.0 and parameters: {'n_rides': 4, 'ride_0': 6, 'ride_1': 0, 'ride_2': 5, 'ride_3': 2}. Best is trial 7 with value: 670.0.
[I 2024-10-18 21:43:55,159] Trial 18 finished with value: 497.0 and parameters: {'n_rides': 7, 'ride_0': 6, 'ride_1': 5, 'ride_2': 2, 'ride_3': 7, 'ride_4': 2, 'ride_5': 6, 'ride_6': 6}. Best is trial

total_time: 170
total_popularity: 135
total_time: 117
total_popularity: 55
total_time: 642
total_popularity: 493
total_time: 247
total_popularity: 186
total_time: 380
total_popularity: 217
total_time: 200
total_popularity: 124
total_time: 590
total_popularity: 437


[I 2024-10-18 21:43:55,235] Trial 21 finished with value: 642.0 and parameters: {'n_rides': 1, 'ride_0': 3}. Best is trial 7 with value: 670.0.
[I 2024-10-18 21:43:55,248] Trial 22 finished with value: 598.0 and parameters: {'n_rides': 1, 'ride_0': 2}. Best is trial 7 with value: 670.0.
[I 2024-10-18 21:43:55,276] Trial 23 finished with value: 653.0 and parameters: {'n_rides': 5, 'ride_0': 3, 'ride_1': 4, 'ride_2': 2, 'ride_3': 5, 'ride_4': 3}. Best is trial 7 with value: 670.0.
[I 2024-10-18 21:43:55,305] Trial 24 finished with value: 652.0 and parameters: {'n_rides': 5, 'ride_0': 5, 'ride_1': 4, 'ride_2': 2, 'ride_3': 5, 'ride_4': 3}. Best is trial 7 with value: 670.0.
[I 2024-10-18 21:43:55,322] Trial 25 finished with value: 663.0 and parameters: {'n_rides': 3, 'ride_0': 4, 'ride_1': 6, 'ride_2': 3}. Best is trial 7 with value: 670.0.
[I 2024-10-18 21:43:55,341] Trial 26 finished with value: 659.0 and parameters: {'n_rides': 3, 'ride_0': 4, 'ride_1': 7, 'ride_2': 3}. Best is trial 7

...

total_time: 681
total_popularity: 588
total_time: 356
total_popularity: 289
total_time: 133
total_popularity: 106
total_time: 133
total_popularity: 106
total_time: 251
total_popularity: 154
total_time: 63
total_popularity: 45
total_time: 135
total_popularity: 88
total_time: 117
total_popularity: 55
total_time: 359
total_popularity: 207


[I 2024-10-18 21:44:26,843] Trial 27 finished with value: 523.5255016347297 and parameters: {'n_rides': 17, 'ride_0': 3, 'ride_1': 3, 'ride_2': 6, 'ride_3': 3, 'ride_4': 7, 'ride_5': 6, 'ride_6': 1, 'ride_7': 4, 'ride_8': 5, 'ride_9': 4, 'ride_10': 7, 'ride_11': 4, 'ride_12': 2, 'ride_13': 2, 'ride_14': 3, 'ride_15': 7, 'ride_16': 0}. Best is trial 9 with value: 647.6630601961675.
[I 2024-10-18 21:44:26,899] Trial 28 finished with value: 532.5309113404784 and parameters: {'n_rides': 12, 'ride_0': 5, 'ride_1': 1, 'ride_2': 3, 'ride_3': 7, 'ride_4': 5, 'ride_5': 1, 'ride_6': 1, 'ride_7': 4, 'ride_8': 7, 'ride_9': 2, 'ride_10': 3, 'ride_11': 0}. Best is trial 9 with value: 647.6630601961675.
[I 2024-10-18 21:44:26,927] Trial 29 finished with value: 539.4026341431218 and parameters: {'n_rides': 4, 'ride_0': 4, 'ride_1': 6, 'ride_2': 5, 'ride_3': 5}. Best is trial 9 with value: 647.6630601961675.
[I 2024-10-18 21:44:26,929] A new study created in memory with name: no-name-5b6eba2b-54ba-4cb4

total_time: 710
total_popularity: 568
total_time: 701
total_popularity: 569
total_time: 175
total_popularity: 108


[I 2024-10-18 21:44:27,049] Trial 11 finished with value: 33.0 and parameters: {'n_rides': 1, 'ride_0': 4, 'lamb_x': 0.9490802962474609}. Best is trial 5 with value: 28.0.
[I 2024-10-18 21:44:27,061] Trial 12 finished with value: 28.0 and parameters: {'n_rides': 1, 'ride_0': 5, 'lamb_x': 0.8948417297512222}. Best is trial 5 with value: 28.0.
[I 2024-10-18 21:44:27,106] Trial 13 finished with value: 324.0 and parameters: {'n_rides': 8, 'ride_0': 5, 'ride_1': 7, 'ride_2': 4, 'ride_3': 4, 'ride_4': 4, 'ride_5': 0, 'ride_6': 7, 'ride_7': 1, 'lamb_x': 1.4423216881535594}. Best is trial 5 with value: 28.0.
[I 2024-10-18 21:44:27,120] Trial 14 finished with value: 28.0 and parameters: {'n_rides': 1, 'ride_0': 5, 'lamb_x': 0.8451425718886445}. Best is trial 5 with value: 28.0.
[I 2024-10-18 21:44:27,180] Trial 15 finished with value: 575.0 and parameters: {'n_rides': 13, 'ride_0': 4, 'ride_1': 1, 'ride_2': 3, 'ride_3': 4, 'ride_4': 7, 'ride_5': 7, 'ride_6': 0, 'ride_7': 2, 'ride_8': 0, 'ride_9

tt_best2: 666


In [27]:
print(f"Best trial: {best_trial.number}")
print(f"Best popularity: {best_trial.value}")
print(f"Parameters: {best_trial.params}")
print("total_time here is:", tt_best)
#print("lamb is",lamb)

Best trial: 9
Best popularity: 647.6630601961675
Parameters: {'n_rides': 14, 'ride_0': 4, 'ride_1': 4, 'ride_2': 6, 'ride_3': 1, 'ride_4': 3, 'ride_5': 6, 'ride_6': 0, 'ride_7': 5, 'ride_8': 0, 'ride_9': 3, 'ride_10': 2, 'ride_11': 2, 'ride_12': 0, 'ride_13': 0}
total_time here is: 666
